In [17]:
import pandas as pd
# see https://www.utf8-chartable.de/unicode-utf8-table.pl?start=9728&number=128&names=-&utf8=dec
import unicodedata

def normalizeC24String(c24String):

  def convertUnicode(string):
    if string[0:2] == "\\u":
      return chr(int(string[2:6], 16))
    else:
      return string

  def replaceUnicode(string):
    utfMapFileUrl = "https://raw.githubusercontent.com/littlecapa/chess24Parser/main/config/mapUTF2PGN.csv"
    mapList = pd.read_csv(utfMapFileUrl, sep=';')
    mapList = mapList.fillna("")
    for index, row in mapList.iterrows():
      string = string.replace(convertUnicode(row["UTF"]), row["pgnStr"])
    return string

  def findPgnToken(string):
    pgnTokenFileUrl = "https://raw.githubusercontent.com/littlecapa/chess24Parser/main/config/pgntoken.csv"
    pgnList = pd.read_csv(pgnTokenFileUrl, sep=';')
    pgnList = pgnList.iloc[pgnList.agg({"string":len}).sort_values('string', ascending=False).index]

    for index, row in pgnList.iterrows():
      string = string.replace(row["string"], row["pgnStr"])
    return string

  c24String = replaceUnicode(c24String)
  c24String = findPgnToken(c24String)

  return c24String


In [18]:
def Test_normalizeC24String():
  # Test Case 1
  c24String ="1. d4 d5+/= 2. c4+- c6-+ 3. ♘f3=/+ ♘f6 4. ♘c3 dxc4 5. a4! ♗f5? 6. e3!! e6?? 7. ♗xc4!? ♗b4?! 8. O-O= O-O 9. ♕e2"
  normString = normalizeC24String(c24String)
  if normString != "1. d4 d5$14 2. c4$18 c6$19 3. Nf3$15 Nf6 4. Nc3 dxc4 5. a4$28 Bf5$29 6. e3$3 e6$4 7. Bxc4$24 Bb4$25 8. O-O$10 O-O 9. Qe2":
    print("Error")
  else:
    print("OK")

In [19]:
Test_normalizeC24String()

OK
